In [18]:
import pandas as pd
from tqdm import tqdm
import time
import re
import requests

In [19]:
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
#options.binary_location = '/usr/local/bin/chromium-browser'

driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver', options = options)
driver.get("https://api.ipify.org/?format=json")
html = driver.page_source
soup = BeautifulSoup(html)
ip = soup.find('pre').text
print('IP from scrape ', ip)

IP from scrape  {"ip":"14.169.99.176"}


In [20]:
def selenium_get_url(url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    return soup

In [22]:
df_sun = pd.read_csv('../Database/ewg_sun.csv')

In [23]:
df_sun['product_score'].value_counts()

03          428
04          243
02          193
07          173
05          166
01          165
06          135
verified     23
08           20
10           18
09            1
Name: product_score, dtype: int64

**Dataset is skew**

In [24]:
df_sun

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img
0,Sun,After Sun Product,https://www.ewg.org/skindeep/products/707220-A...,ATTITUDE,"little ones Aftersun melt-in Gel, Calendula, T...",verified,https://static.ewg.org/skindeep_images/7072/70...
1,Sun,After Sun Product,https://www.ewg.org/skindeep/products/671517-A...,AROMATICA,95% Organic Aloe Vera Gel,verified,https://static.ewg.org/skindeep_images/6715/67...
2,Sun,After Sun Product,https://www.ewg.org/skindeep/products/823174-O...,Olita,"After Sun Body Serum, Fragrance Free",verified,https://static.ewg.org/skindeep_images/8231/82...
3,Sun,After Sun Product,https://www.ewg.org/skindeep/products/710965-V...,Vika's Essentials,'Prodigal Sun' Oil Blend,verified,https://static.ewg.org/skindeep_images/7109/71...
4,Sun,After Sun Product,https://www.ewg.org/skindeep/products/899338-O...,Olita,"After Sun Body Serum, Soothing Citrus",verified,https://static.ewg.org/skindeep_images/8993/89...
...,...,...,...,...,...,...,...
1560,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/929454-C...,COOLA,"Classic Body Sunscreen Spray, Peach Blossom, S...",10,https://static.ewg.org/skindeep_images/9294/92...
1561,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/929145-C...,CVS,"Health Ultra Protection Sun Lotion, SPF 100",10,https://static.ewg.org/skindeep_images/9291/92...
1562,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/929320-A...,Australian Gold,"Botanical Natural Sunscreen Spray, SPF 70",10,https://static.ewg.org/skindeep_images/9293/92...
1563,Sun,Beach & Sport Sunscreen,https://www.ewg.org/skindeep/products/928941-B...,Banana Boat,"Ultra Sport Sunscreen Lotion, SPF 100",10,https://static.ewg.org/skindeep_images/9289/92...


In [25]:
df_sun.isna().sum()

cat_name          0
subcat_name       0
product_url       0
product_brand    17
product_name      0
product_score     0
product_img       0
dtype: int64

In [26]:
df_sun.loc[0, 'product_url']

'https://www.ewg.org/skindeep/products/707220-ATTITUDE_little_ones_Aftersun_meltin_Gel_Calendula_Tutti_Fruti/'

In [27]:
list_link = df_sun['product_url'].to_list()

In [28]:
len(list_link)

1565

In [29]:
soup = selenium_get_url('https://www.ewg.org/skindeep/products/701958-Soap_for_Goodness_Sake_Handmade_Soap_Olive_Oil_I/')

In [30]:
soup

<html class="" lang="en"><head><script src="https://z.moatads.com/addthismoatframe568911941483/moatframe.js" type="text/javascript"></script>
<meta charset="utf-8"/>
<title>EWG Skin Deep® | Soap for Goodness Sake Handmade Soap, Olive Oil I Rating</title>
<meta content="EWG’s Skin Deep® Cosmetics Database Rating for Soap for Goodness Sake Handmade Soap, Olive Oil I." name="description"/>
<link href="https://www.ewg.org/skindeep/products/701958-Soap_for_Goodness_Sake_Handmade_Soap_Olive_Oil_I" rel="canonical"/>
<meta content="website" property="og:type"/>
<meta content="EWG" property="og:site_name"/>
<meta content="EWG Skin Deep® | Soap for Goodness Sake Handmade Soap, Olive Oil I Rating" property="og:title"/>
<meta content="EWG’s Skin Deep® Cosmetics Database Rating for Soap for Goodness Sake Handmade Soap, Olive Oil I." property="og:description"/>
<meta content="https://www.ewg.org/skindeep/products/701958-Soap_for_Goodness_Sake_Handmade_Soap_Olive_Oil_I" property="og:url"/>
<meta cont

In [48]:
get_ing = soup.find('section', {'id': 'label-information'})
ing = get_ing.find('p').get_text()

In [49]:
print(ing)

Saponified Organic Extra Virgin Olive Oil (Sodium Olivate), Distilled Water, Cinnamomum Zeylanicum (Cinnamon) Leaf Oil,


In [31]:
ingredient_list = []
for link in tqdm(list_link):
    soup = selenium_get_url(link)
    
    get_ing = soup.find('section', {'id': 'label-information'})
    ing = get_ing.find('p').get_text() if get_ing else 'No ingredient list'
    ingredient_list.append(ing)
    
    time.sleep(1)

100%|██████████| 1565/1565 [1:26:49<00:00,  3.33s/it]


In [32]:
len(ingredient_list)

1565

In [14]:
df_sun['ingredient_list'] = ingredient_list

In [33]:
df_sun.to_csv('ewg_sun_full_1565.csv', index=False)

In [34]:
df_sun.head()

,cat_name,subcat_name,product_url,product_brand,product_name,product_score,product_img
0,Sun,After Sun Product,https://www.ewg.org/skindeep/products/707220-A...,ATTITUDE,"little ones Aftersun melt-in Gel, Calendula, T...",verified,https://static.ewg.org/skindeep_images/7072/70...
1,Sun,After Sun Product,https://www.ewg.org/skindeep/products/671517-A...,AROMATICA,95% Organic Aloe Vera Gel,verified,https://static.ewg.org/skindeep_images/6715/67...
2,Sun,After Sun Product,https://www.ewg.org/skindeep/products/823174-O...,Olita,"After Sun Body Serum, Fragrance Free",verified,https://static.ewg.org/skindeep_images/8231/82...
3,Sun,After Sun Product,https://www.ewg.org/skindeep/products/710965-V...,Vika's Essentials,'Prodigal Sun' Oil Blend,verified,https://static.ewg.org/skindeep_images/7109/71...
4,Sun,After Sun Product,https://www.ewg.org/skindeep/products/899338-O...,Olita,"After Sun Body Serum, Soothing Citrus",verified,https://static.ewg.org/skindeep_images/8993/89...


In [35]:
df_sun['product_score'].value_counts()

03          428
04          243
02          193
07          173
05          166
01          165
06          135
verified     23
08           20
10           18
09            1
Name: product_score, dtype: int64